In [ ]:
from pavlov import *
monitor(-1)

In [ ]:
from pavlov import stats
stats.plot(['*wry-sights*', '*prize-remedy*', '*hairy-surge*', -1], 'elo-mohex', 'μ', skip=1, head=240, fill=True)

In [ ]:
from pavlov import stats
stats.plot(['*wry-sights*', '*prize-remedy*', '*hairy-surge*', '*taut-runway*', -1], 'rel-entropy.policy', skip=0, head=240, fill=True)